In [ ]:
WINDOW_SIZE = 1275
WINDOW_STEP = 21
output_name= "feature_120_1.csv"

In [ ]:
import multiprocessing
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, iqr
from scipy.fft import fft
from scipy.stats import entropy
import pywt

# ======================== Frequency-domain features ======================== #
def get_fft_stats(df, columns=['steering', 'throttle', 'brake']):
    """
    Compute FFT-based frequency-domain features for selected signals.
    """
    features = {}

    for col in columns:
        if col not in df.columns:
            continue

        signal = df[col].dropna().values  # Remove NaN values

        if len(signal) < 2:
            continue  # Ignore too-short signals

        # 1. Perform FFT and keep only positive frequency components
        fft_vals = np.abs(fft(signal))
        fft_vals = fft_vals[:len(fft_vals)//2]

        # 2. Spectral statistics
        features[f'FFT_{col}_mean'] = np.mean(fft_vals)
        features[f'FFT_{col}_std'] = np.std(fft_vals)
        features[f'FFT_{col}_max'] = np.max(fft_vals)
        features[f'FFT_{col}_min'] = np.min(fft_vals)
        features[f'FFT_{col}_energy'] = np.sum(fft_vals ** 2)

        # 3. Spectral entropy
        psd = fft_vals ** 2
        psd_norm = psd / np.sum(psd) if np.sum(psd) != 0 else psd
        features[f'FFT_{col}_entropy'] = entropy(psd_norm)

    return features


# ======================== Wavelet transform features ======================= #
def extract_wavelet_features(df, columns=['steering', 'throttle', 'brake'], wavelet='db4', level=3):
    """
    Extract wavelet-based signal energy, mean, entropy for each decomposition level.
    """
    features = {}

    for col in columns:
        if col not in df.columns:
            continue

        signal = df[col].dropna().values
        if len(signal) < 2:
            continue

        coeffs = pywt.wavedec(signal, wavelet, level=level)

        for i, coeff in enumerate(coeffs):
            prefix = f'{col}_L{i}'
            features[f'WVL_{prefix}_mean'] = np.mean(coeff)
            features[f'WVL_{prefix}_std'] = np.std(coeff)
            features[f'WVL_{prefix}_max'] = np.max(coeff)
            features[f'WVL_{prefix}_energy'] = np.sum(np.square(coeff))
            coeff_norm = np.square(coeff) / np.sum(np.square(coeff)) if np.sum(np.square(coeff)) > 0 else coeff
            features[f'WVL_{prefix}_entropy'] = entropy(coeff_norm)

    return features


# ======================== Handcrafted pedal features ====================== #
def extract_pedal(pedal_df: pd.DataFrame):
    """
    Extract handcrafted driving maneuver features from pedal signals.
    """
    res = {
        "LAST_steer_sum": np.nan,
        "LAST_steer_abs_sum": np.nan,
        "LAST_steer_flucuate_times": np.nan,
        "LAST_steer_sum_per_fulct": np.nan,
        "LAST_steer_max_fluct": np.nan,
        "LAST_steer_mean_fluct_speed": np.nan,
        "LAST_steer_max_fluct_speed": np.nan,
        "LAST_throttle_duration": np.nan,
        "LAST_brake_duration": np.nan,
        "LAST_throttle_brake_ratio": np.nan,
        "LAST_brake_times": np.nan,
        "LAST_throttle_auc": np.nan,
    }

    if pedal_df.shape[0] == 0:
        return res
    
    pedal_df = pedal_df.copy()
    pedal_df["throttle"] = -pedal_df["throttle"] + 32767.0
    pedal_df["brake"] = -pedal_df["brake"] + 32767.0

    res["LAST_steer_sum"] = pedal_df["steering"].sum()
    res["LAST_steer_abs_sum"] = np.abs(pedal_df["steering"]).sum()

    try:
        pedal_df["LAST_pass_zero"] = (pedal_df["steering"] * pedal_df["steering"].shift(1)).map(lambda x: 1 if x <= 0 else 0)
        res["LAST_steer_flucuate_times"] = pedal_df["LAST_pass_zero"].sum()
        res["LAST_steer_sum_per_fulct"] = 0 if res["LAST_steer_flucuate_times"] == 0 else res["LAST_steer_abs_sum"] / res["LAST_flucuate_times"]
    except:
        res["LAST_flucuate_times"] = 0
        res["LAST_steer_sum_per_fulct"] = 0

    res["LAST_steer_max_fluct"] = np.abs(pedal_df["steering"]).max()
    res["LAST_steer_mean_fluct_speed"] = np.nanmean(np.abs(pedal_df["steering_derivative"]))
    res["LAST_steer_max_fluct_speed"] = np.abs(pedal_df["steering_derivative"]).max()
    
    pedal_df["dura"] = pedal_df["timestamp"].shift(-1) - pedal_df["timestamp"]
    res["LAST_throttle_duration"] = pedal_df[~(pedal_df["throttle"] == 0)]["dura"].sum()
    res["LAST_brake_duration"] = pedal_df[~(pedal_df["brake"] == 0)]["dura"].sum()
    res["LAST_throttle_brake_ratio"] = res["LAST_throttle_duration"] / (res["LAST_brake_duration"] + 0.01)

    _temp_times = 0
    _last_idx = pedal_df.index[0]
    for idx, row in pedal_df[pedal_df["brake"] == 0].iterrows():
        if not (idx - _last_idx == 1 or idx - _last_idx == 0):
            _temp_times += 1
        _last_idx = idx
    res["LAST_brake_times"] = _temp_times
    res["LAST_throttle_auc"] = pedal_df["throttle"].sum()
    return res


# ======================== New pedal features ============================== #
def extract_new_pedal_feature(pedal_df: pd.DataFrame):
    """
    New pedal dynamics and spectral feature extraction.
    """
    res = {
        "NewPedal_steering_Change_quantile_mean": np.nan,
        "NewPedal_steering_fft_Kurtosis": np.nan,
        "NewPedal_steering_unique_value_ratio": np.nan,
        "NewPedal_steering_FFT_Aggregated_Centroid": np.nan
    }

    if pedal_df.shape[0] == 0:
        return res

    # Steering absolute change rate mean
    steering_diff = pedal_df["steering"].diff()
    abs_change = steering_diff.abs()
    time_diff = pedal_df["timestamp"].diff().replace(0, np.nan)
    steering_rate = abs_change / time_diff
    res["NewPedal_steering_Change_quantile_mean"] = steering_rate.mean()

    # Spectral kurtosis
    steering_signal = pedal_df["steering"].dropna().values
    fft_magnitude = np.abs(fft(steering_signal))
    res["NewPedal_steering_fft_Kurtosis"] = kurtosis(fft_magnitude, fisher=False)

    # Unique value ratio
    res["NewPedal_steering_unique_value_ratio"] = pedal_df["steering"].nunique() / len(pedal_df["steering"])

    # FFT centroid frequency
    n = len(steering_signal)
    fft_vals = fft(steering_signal)
    fft_mag = np.abs(fft_vals)[:n//2]
    freqs = np.fft.fftfreq(n, d=1.0/21.75)[:n//2]
    res["NewPedal_steering_FFT_Aggregated_Centroid"] = (
        np.sum(freqs * fft_mag) / np.sum(fft_mag) if np.sum(fft_mag) > 0 else 0
    )
    return res


# ======================== New speed features ============================== #
def extract_new_speed_feature(speed_df: pd.DataFrame):
    """
    New features describing high-speed events, nonlinearity, and acceleration variability.
    """
    res = {
        "NewSpeed_90%_quantile": np.nan,
        "NewSpeed_C3": np.nan,
        "NewSpeed_change_quantile_variance": np.nan
    }
    if speed_df.shape[0] == 0:
        return res

    speed_series = speed_df["speed"]
    n = len(speed_series)

    q90 = speed_series.quantile(0.9)
    res["NewSpeed_90%_quantile"] = len(speed_series[speed_series > q90]) / n

    speed_values = speed_series.dropna().values
    n = len(speed_values)
    mu = np.mean(speed_values)

    sum_c3 = sum(speed_values[i] * speed_values[i-1] * speed_values[i-2]
                 for i in range(2, n))
    res["NewSpeed_C3"] = (sum_c3 / (n - 2)) - mu**3

    abs_diff = np.abs(speed_series.diff().dropna())
    res["NewSpeed_change_quantile_variance"] = np.var(abs_diff)

    return res


# ======================== Eye-gaze geometry =============================== #
from shapely.geometry import Point, box
from shapely.ops import unary_union

def compute_gaze_area(data: pd.DataFrame, radius=25):
    """
    Compute union area of gaze points buffered to circles clipped inside screen.
    """
    def create_circle(x, y, radius):
        screen_box = box(0, 0, 1920, 1080)
        circle = Point(x, y).buffer(radius)
        return circle.intersection(screen_box)

    data['circle'] = data.apply(lambda row: create_circle(row['Gaze point X'], row['Gaze point Y'], radius), axis=1)
    union_area = unary_union(data['circle'].tolist())
    return union_area.area


# ======================== Eye movement segmentation features ============== #
def extract_eye_movement_type(eye_df):
    """
    Extract fixation/saccade counts and related amplitude/velocity statistics.
    """
    eye_df = eye_df.copy()
    res = {
        "GE_saccade_times": np.nan,
        "GE_fixation_times": np.nan,
        "GE_saccade_duration_mean": np.nan,
        "GE_saccade_duration_std": np.nan,
        "GE_saccade_duration_005quantiles": np.nan,
        "GE_saccade_duration_095quantiles": np.nan,
        "GE_saccade_duration_skewness": np.nan,
        "GE_saccade_duration_kurtosis": np.nan,
        "GE_fixation_duration_mean": np.nan,
        "GE_fixation_duration_std": np.nan,
        "GE_fixation_duration_005quantiles": np.nan,
        "GE_fixation_duration_095quantiles": np.nan,
        "GE_fixation_duration_skewness": np.nan,
        "GE_fixation_duration_kurtosis": np.nan,
        "GE_saccade_amplitude_mean": np.nan,
        "GE_saccade_amplitude_std": np.nan,
        "GE_saccade_amplitude_005quantiles": np.nan,
        "GE_saccade_amplitude_095quantiles": np.nan,
        "GE_saccade_amplitude_skewness": np.nan,
        "GE_saccade_amplitude_kurtosis": np.nan,
        "GE_saccade_peak_v_mean": np.nan,
        "GE_saccade_peak_v_std": np.nan,
        "GE_saccade_peak_v_005quantiles": np.nan,
        "GE_saccade_peak_v_095quantiles": np.nan,
        "GE_saccade_peak_v_skewness": np.nan,
        "GE_saccade_peak_v_kurtosis": np.nan,
        "GE_saccade_mean_v_mean": np.nan,
        "GE_saccade_mean_v_std": np.nan,
        "GE_saccade_mean_v_005quantiles": np.nan,
        "GE_saccade_mean_v_095quantiles": np.nan,
        "GE_saccade_mean_v_skewness": np.nan,
        "GE_saccade_mean_v_kurtosis": np.nan,
    }

    eye_df['dt'] = eye_df['timestamp'].diff()
    eye_df['dx'] = eye_df['Gaze point X'].diff()
    eye_df['dy'] = eye_df['Gaze point Y'].diff()
    eye_df['velocity'] = np.sqrt(eye_df['dx']**2 + eye_df['dy']**2) / eye_df['dt']

    eye_df['segment_id'] = (eye_df['Eye movement type'] != eye_df['Eye movement type'].shift()).cumsum()

    results = []
    for seg_id, group in eye_df.groupby('segment_id'):
        seg_type = group['Eye movement type'].iloc[0]
        start_time = group['timestamp'].iloc[0]
        end_time = group['timestamp'].iloc[-1]
        duration = end_time - start_time
        
        if seg_type == 'Fixation':
            results.append({'type': 'Fixation', 'duration': duration})
        elif seg_type == 'Saccade':
            amplitude = np.sqrt((group['Gaze point X'].iloc[-1] - group['Gaze point X'].iloc[0])**2 +
                                (group['Gaze point Y'].iloc[-1] - group['Gaze point Y'].iloc[0])**2)
            results.append({
                'type': 'Saccade',
                'duration': duration,
                'amplitude': amplitude,
                'peak_velocity': group['velocity'].max(),
                'mean_velocity': group['velocity'].mean()
            })

    res_df = pd.DataFrame(results)
    if res_df.empty:
        return res

    fix_df = res_df[res_df['type'] == 'Fixation']
    sac_df = res_df[res_df['type'] == 'Saccade']

    res["GE_fixation_times"] = len(fix_df)
    res["GE_saccade_times"] = len(sac_df)

    if not fix_df.empty:
        dur = fix_df['duration']
        res["GE_fixation_duration_mean"] = dur.mean()
        res["GE_fixation_duration_std"] = dur.std()
        res["GE_fixation_duration_005quantiles"] = dur.quantile(0.05)
        res["GE_fixation_duration_095quantiles"] = dur.quantile(0.95)
        res["GE_fixation_duration_skewness"] = skew(dur, nan_policy='omit')
        res["GE_fixation_duration_kurtosis"] = kurtosis(dur, nan_policy='omit')

    if not sac_df.empty:
        dur = sac_df['duration']
        amp = sac_df['amplitude']
        pv = sac_df['peak_velocity']
        mv = sac_df['mean_velocity']

        res["GE_saccade_duration_mean"] = dur.mean()
        res["GE_saccade_duration_std"] = dur.std()
        res["GE_saccade_duration_005quantiles"] = dur.quantile(0.05)
        res["GE_saccade_duration_095quantiles"] = dur.quantile(0.95)
        res["GE_saccade_duration_skewness"] = skew(dur, nan_policy='omit')
        res["GE_saccade_duration_kurtosis"] = kurtosis(dur, nan_policy='omit')

        res["GE_saccade_amplitude_mean"] = amp.mean()
        res["GE_saccade_amplitude_std"] = amp.std()
        res["GE_saccade_amplitude_005quantiles"] = amp.quantile(0.05)
        res["GE_saccade_amplitude_095quantiles"] = amp.quantile(0.95)
        res["GE_saccade_amplitude_skewness"] = skew(amp, nan_policy='omit')
        res["GE_saccade_amplitude_kurtosis"] = kurtosis(amp, nan_policy='omit')

        res["GE_saccade_peak_v_mean"] = pv.mean()
        res["GE_saccade_peak_v_std"] = pv.std()
        res["GE_saccade_peak_v_005quantiles"] = pv.quantile(0.05)
        res["GE_saccade_peak_v_095quantiles"] = pv.quantile(0.95)
        res["GE_saccade_peak_v_skewness"] = skew(pv, nan_policy='omit')
        res["GE_saccade_peak_v_kurtosis"] = kurtosis(pv, nan_policy='omit')

        res["GE_saccade_mean_v_mean"] = mv.mean()
        res["GE_saccade_mean_v_std"] = mv.std()
        res["GE_saccade_mean_v_005quantiles"] = mv.quantile(0.05)
        res["GE_saccade_mean_v_095quantiles"] = mv.quantile(0.95)
        res["GE_saccade_mean_v_skewness"] = skew(mv, nan_policy='omit')
        res["GE_saccade_mean_v_kurtosis"] = kurtosis(mv, nan_policy='omit')

    return res


# ======================== Eye movement general features =================== #
def extract_eye(eye_df):
    """
    Compute eye movement speed statistics.
    """
    res = {
        "LASTE_eye_speed_x_mean": np.nan,
        "LASTE_eye_speed_y_mean": np.nan,
        "LASTE_eye_speed_eye_mean": np.nan,
        "LASTE_eye_speed_x_max": np.nan,
        "LASTE_eye_speed_y_max": np.nan,
        "LASTE_eye_speed_eye_max": np.nan,
    }

    if eye_df.shape[0] == 0:
        return res

    x_speed = np.abs(eye_df["Gaze point X_derivative"])
    y_speed = np.abs(eye_df["Gaze point Y_derivative"])
    all_speed = np.abs(eye_df["eyemovement_speed"])

    res["LASTE_eye_speed_x_mean"] = np.nanmean(x_speed) if len(x_speed) > 0 else np.nan
    res["LASTE_eye_speed_y_mean"] = np.nanmean(y_speed) if len(y_speed) > 0 else np.nan
    res["LASTE_eye_speed_eye_mean"] = np.nanmean(all_speed)

    res["LASTE_eye_speed_x_max"] = np.max(x_speed)
    res["LASTE_eye_speed_y_max"] = np.max(y_speed)
    res["LASTE_eye_speed_eye_max"] = np.max(all_speed)

    return res


# ======================== Basic statistical feature calculators =========== #
def get_stats(data, key_suffix: str = None, feature_type: str = None):
    """
    Return basic statistics for a signal: mean, std, percentiles, skewness, kurtosis.
    """
    results = {
        'mean': np.nan,
        'std': np.nan,
        'q5': np.nan,
        'q95': np.nan,
        'skewness': np.nan,
        'kurtosis': np.nan,
    }

    if len(data) > 0:
        results['mean'] = np.mean(data)
        results['std'] = np.std(data)
        results['q5'] = np.quantile(data, 0.05)
        results['q95'] = np.quantile(data, 0.95)
        results['skewness'] = skew(data) if np.std(data) >= 1e-8 else np.nan
        results['kurtosis'] = kurtosis(data) if np.std(data) >= 1e-8 else np.nan

    prefix_map = {
        'pedal': 'STATSP_',
        'speed': 'STATSS_',
        'eyemovement': 'STATSE_',
        'head': 'STATSH_'
    }

    prefix = prefix_map.get(feature_type, '')
    if key_suffix is not None:
        results = {f"{prefix}{key_suffix}_{k}": v for k, v in results.items()}
    else:
        results = {f"{prefix}{k}": v for k, v in results.items()}

    return results


def get_stats_new(data, key_suffix: str = None, feature_type: str = None):
    """
    Return advanced signal statistics: peak-to-peak, RMS, energy, IQR, etc.
    """
    results = {
        'ptp': np.nan,
        'median': np.nan,
        'energy': np.nan,
        'rms': np.nan,
        'lineintegral': np.nan,
        'n_sign_changes': np.nan,
        'iqr': np.nan,
        'iqr_5_95': np.nan
    }

    if len(data) > 0:
        results['ptp'] = np.ptp(data)
        results['median'] = np.median(data)
        results['energy'] = np.sum(data ** 2)
        results['rms'] = np.sqrt(np.sum(data ** 2) / len(data))
        results['lineintegral'] = np.abs(np.diff(data)).sum()
        results['n_sign_changes'] = np.sum(np.diff(np.sign(data)) != 0)
        results['iqr'] = np.subtract(*np.nanpercentile(data, [75, 25]))
        results['iqr_5_95'] = np.subtract(*np.nanpercentile(data, [95, 5]))

    prefix_map = {
        'pedal': 'STATSPNEW_',
        'speed': 'STATSSNEW_',
        'eyemovement': 'STATSENEW_',
        'head': 'STATSHNEW_'
    }

    prefix = prefix_map.get(feature_type, '')
    if key_suffix is not None:
        results = {f"{prefix}{key_suffix}_{k}": v for k, v in results.items()}
    else:
        results = {f"{prefix}{k}": v for k, v in results.items()}

    return results


In [ ]:
# Sliding-window feature extraction

def get_sliding_window(data: pd.DataFrame, speed_data: pd.DataFrame,
                       eyemovement_data: pd.DataFrame, start_end_list):
    """
    Extract aggregated features for a given time window.

    Parameters
    ----------
    data : pd.DataFrame
        Pedal-related time-series window subset.
    speed_data : pd.DataFrame
        Speed-related time-series window subset.
    eyemovement_data : pd.DataFrame
        Eye-movement-related time-series window subset.
    start_end_list : [float, float]
        [window_start_timestamp, window_end_timestamp]

    Returns
    -------
    dict
        A dictionary containing all aggregated feature values.
    """
    min_timestamp, max_timestamp = start_end_list

    results = {"datetime": min_timestamp}

    # Select samples inside the time window
    relevant_data = data[(data["timestamp"] > min_timestamp) &
                         (data["timestamp"] < max_timestamp)]
    relevant_speed_data = speed_data[(speed_data["timestamp"] > min_timestamp) &
                                     (speed_data["timestamp"] < max_timestamp)]
    relevant_eyemovement_data = eyemovement_data[(eyemovement_data["timestamp"] > min_timestamp) &
                                                 (eyemovement_data["timestamp"] < max_timestamp)]

    # ======================= Pedal features ======================= #
    for column in relevant_data.columns:
        if column in ["timestamp", "person_id"]:
            continue
        stats = get_stats(relevant_data[column], column, "pedal")
        results.update(stats)
        stats_new = get_stats_new(relevant_data[column], column, "pedal")
        results.update(stats_new)

    # Frequency-domain + Wavelet + Handcrafted pedal features
    results.update(get_fft_stats(relevant_data))
    results.update(extract_wavelet_features(relevant_data))
    results.update(extract_pedal(relevant_data))
    results.update(extract_new_pedal_feature(relevant_data))

    # ======================= Speed features ======================= #
    results.update(extract_new_speed_feature(relevant_speed_data))

    for column in relevant_speed_data.columns:
        if column in ["timestamp", "person_id"]:
            continue
        stats = get_stats(relevant_speed_data[column], column, "speed")
        results.update(stats)
        stats_new = get_stats_new(relevant_speed_data[column], column, "speed")
        results.update(stats_new)

    # ======================= Eye movement features ======================= #
    for column in relevant_eyemovement_data.columns:
        if column in [
            "timestamp", "person_id", "Eye movement type",
            "Eye position left X (DACSmm)", "Eye position left Y (DACSmm)",
            "Eye position left Z (DACSmm)", "Eye position right X (DACSmm)",
            "Eye position right Y (DACSmm)", "Eye position right Z (DACSmm)"
        ]:
            continue

        feature_type = "head" if column.startswith("Head") else "eyemovement"

        stats = get_stats(relevant_eyemovement_data[column], column, feature_type)
        results.update(stats)
        stats_new = get_stats_new(relevant_eyemovement_data[column], column, feature_type)
        results.update(stats_new)

    # Eye movement segmentation features
    results.update(extract_eye_movement_type(relevant_eyemovement_data))
    # Optionally: results.update(extract_eye(relevant_eyemovement_data))

    return results


def get_features(data: pd.DataFrame, speed_data: pd.DataFrame,
                 eyemovement_data: pd.DataFrame, num_cores: int = 0,
                 start_row: int = 0):
    """
    Sliding-window feature extraction over entire dataset.

    Parameters
    ----------
    data : pd.DataFrame
        Pedal dataset with timestamp.
    speed_data : pd.DataFrame
        Speed dataset aligned by timestamp.
    eyemovement_data : pd.DataFrame
        Eye tracking dataset aligned by timestamp.
    num_cores : int
        Number of CPU cores for future parallel acceleration (unused now).
    start_row : int
        Not used here but reserved for future functionality.

    Returns
    -------
    pd.DataFrame
        Feature matrix indexed by window start timestamp.
    """

    input_data = data.copy()
    input_speed_data = speed_data.copy()
    input_eyemovement_data = eyemovement_data.copy()

    total_rows = len(input_data)
    windows = []

    # ⚠️ These must be already defined from config (unchanged behavior)
    global WINDOW_SIZE_S, STEP_SIZE_S
    WINDOW_SIZE = WINDOW_SIZE_S
    WINDOW_STEP = STEP_SIZE_S

    # Build sliding windows
    for start in range(0, total_rows - WINDOW_SIZE, WINDOW_STEP):
        start_timestamp = float(input_data.iloc[start]["timestamp"])
        end_timestamp = float(input_data.iloc[start + WINDOW_SIZE]["timestamp"])
        windows.append([start_timestamp, end_timestamp])

    if not windows:
        return []

    results = [
        get_sliding_window(input_data, input_speed_data, input_eyemovement_data, w)
        for w in windows
    ]

    results_df = pd.DataFrame([r for r in results if r is not None])
    results_df.set_index("datetime", inplace=True)
    results_df.sort_index(inplace=True)

    return results_df


In [ ]:
import pandas as pd
from config.config_loader import load_config
import os

CONFIG_DATA = load_config(os.path.join("config", "feature_engineering_config.json"))
NC_number = CONFIG_DATA["NC_number"]
PD_number = CONFIG_DATA["PD_number"]
result = pd.DataFrame()

WINDOW_SIZE_LIST = [1275*2]  #120
size_list =['120']
WINDOW_STEP_LIST = [21]
step_list = ['1']
output_name= f"feature_{size_list[0]}_{step_list[0]}.csv"

for type in ['PD','NC']:
    for ex in ['ex1','ex2']:

        if type == 'PD':
            person_amount = PD_number
        else:
            person_amount = NC_number

        for person_num in range(person_amount):
            print("在处理第"+str(person_num+1)+"个文件")

            data_type = "pedal"        
            pedal_data = pd.read_csv(
                                f"sliced_data/{type}/{type}{person_num + 1}/whole_{data_type}_{ex}.csv"
                            )

            data_type = "speed"        
            speed_data = pd.read_csv(
                                f"sliced_data/{type}/{type}{person_num + 1}/whole_{data_type}_{ex}.csv"
                            )
            
            data_type = "eyemovement"        
            eyemovement_data = pd.read_csv(
                                f"sliced_data/{type}/{type}{person_num + 1}/whole_{data_type}_{ex}.csv"
                            )

            res = get_features(pedal_data,speed_data,eyemovement_data,0,0)

            res = pd.concat([
                res,
                pd.DataFrame({
                    'participant': [f"{type} P{person_num + 1}"] * len(res),
                    'experiment': [ex] * len(res),
                    'label':1 if type =='PD' else 0
                }, index=res.index)
            ], axis=1)

            result = pd.concat([result, res], axis=0)

result.to_csv(output_name)